In [3]:
from random import random
import numpy as np
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras import layers
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
import pickle
from keras.models import Model
import os
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
os.environ["CUDA_VISIBLE_DEVICES"]="1"


Using TensorFlow backend.


In [ ]:
DATA = "ARTINT"

In [4]:
!nvidia-smi

Sun Jan 13 13:45:28 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0  On |                  N/A |
| 42%   64C    P2    83W / 250W |   9083MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 23%   36C    P8    16W / 250W |    506MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [5]:
train_filename = pickle.load(open(DATA+"_train_fulltext_filename_final.p", "rb"))
train_use_vector = np.load(DATA+"_train_fulltext_use_vector_final.npy")
test_filename = pickle.load(open(DATA+"_test_fulltext_filename_final.p", "rb"))
test_use_vector = np.load(DATA+"_test_fulltext_use_vector_final.npy")



In [6]:
!nvidia-smi

Sun Jan 13 13:46:10 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0  On |                  N/A |
| 41%   64C    P2   145W / 250W |   9083MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 23%   36C    P8    16W / 250W |    506MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [7]:
y = []
c = 0
for i in train_filename:
    c = c + 1
    if c%500 == 0:
        print(c)
    if "pos_" in i:
        y.append(1.0)
    else :
        y.append(0.0)

500
1000
1500
2000
2500
3000


In [8]:
y_test = []
c = 0
for i in test_filename:
    c = c + 1
    if c%500 == 0:
        print(c)
    if "pos_" in i:
        y_test.append(1.0)
    else :
        y_test.append(0.0)

500


In [9]:
print(train_use_vector.shape)
print(test_use_vector.shape)

(3056, 500, 512)
(768, 500, 512)


In [10]:

n_timesteps = train_use_vector[0].shape[0]
# define LSTM
model = Sequential()
model.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(n_timesteps, 512)))
model.add(Bidirectional(LSTM(50, return_sequences=True)))
model.add(Bidirectional(LSTM(50)))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(100, activation = "relu"))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(4000, activation = "relu"))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))

# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))

model.summary()
checkpoint = ModelCheckpoint(DATA+'model-full-text-best.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 500, 200)          490400    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 500, 100)          100400    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100)               60400     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
__________

In [ ]:
results = model.fit( train_use_vector,y, epochs= 10, batch_size = 64,validation_data=(test_use_vector,y_test),callbacks=[checkpoint])


Train on 3056 samples, validate on 768 samples
Epoch 1/10
3056/3056 [==============================] - 289s 95ms/step - loss: 0.4721 - acc: 0.7866 - val_loss: 0.3754 - val_acc: 0.8372

Epoch 00001: val_loss improved from inf to 0.37540, saving model to model-full-text-best.h5
Epoch 2/10
3056/3056 [==============================] - 284s 93ms/step - loss: 0.3292 - acc: 0.8603 - val_loss: 0.2842 - val_acc: 0.8724

Epoch 00002: val_loss improved from 0.37540 to 0.28416, saving model to model-full-text-best.h5
Epoch 3/10
3056/3056 [==============================] - 294s 96ms/step - loss: 0.2714 - acc: 0.8865 - val_loss: 0.2403 - val_acc: 0.8919

Epoch 00003: val_loss improved from 0.28416 to 0.24026, saving model to model-full-text-best.h5
Epoch 4/10
1024/3056 [=========>....................] - ETA: 2:55 - loss: 0.2540 - acc: 0.8955

In [ ]:
model.layers

In [ ]:
#model.load_weights('model-best.h5')

new_model = Model(inputs = model.inputs, outputs = model.layers[-3].output)

new_model.load_weights(DATA+'model-full-text-best.h5', by_name = True)
train_prediction = new_model.predict(train_use_vector)
test_prediction = new_model.predict(test_use_vector)

In [ ]:
np.save(DATA+'_train_full_text_modality_feature',train_prediction)

np.save(DATA+'_test_full_text_modality_feature',test_prediction)


In [ ]:
predicted =  model.predict(test_use_vector)
for i in range(len(predicted)):
    if predicted[i] >= 0.5:
        predicted[i] = 1.0
    else :
        predicted[i] = 0.0

In [ ]:
print("fscore",f1_score(y_test, predicted, average="macro"))
print("precision",precision_score(y_test, predicted, average="macro"))
print("recall",recall_score(y_test, predicted, average="macro"))    

x = classification_report(y_test,predicted)
print(x)

In [ ]:
model.save(DATA+'onlyfulltextmodel')